In [119]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, ICRS, Galactic, GalacticLSR, Galactocentric
import astropy.units as u
import galpy
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014

time = np.linspace(0,-50,100)*u.Myr

df_heiles = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/galactic_motions_plotly/data/maria_clusters_heiles_shell.csv')
df_split = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/galactic_motions_plotly/data/maria_clusters_split.csv')
df_young = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/galactic_motions_plotly/data/maria_clusters_young.csv')
df_older = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/galactic_motions_plotly/data/maria_clusters_older.csv')
df_rw = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/galactic_motions_plotly/data/maria_clusters_rw_joao.csv')
df_rw['rv_weighted_med'] = df_rw['rv_weighted']

In [120]:
def compute_orbit(df, time):
    icrs = SkyCoord(ra=df.ra.values * u.deg,
                    dec=df.dec.values * u.deg,
                    distance=(1000 / df.parallax.values) * u.pc,
                    pm_ra_cosdec = df.pmra.values * (u.mas/u.yr),
                    pm_dec = df.pmdec.values * (u.mas/u.yr),
                    radial_velocity = df.rv_weighted_med.values * (u.km/u.s)
                    )
    orbit = Orbit(vxvv=icrs)
    potential = MWPotential2014
    orbit.integrate(time, pot=potential)

    # E = np.median(orbit.E(time), axis = 1)
    # Er = np.median(orbit.ER(time), axis = 1)
    # Ez = np.median(orbit.Ez(time), axis = 1)

    # L = np.median(orbit.L(time), axis = 1)
    # LcE = np.median(orbit.LcE(time), axis = 1)
    # Lz = np.median(orbit.Lz(time), axis = 1)

    E_i = orbit.E(time[0])
    Er_i = orbit.ER(time[0])
    Ez_i = orbit.Ez(time[0])
    Lz_i = orbit.Lz(time[0])

    E_f = orbit.E(time[71])
    Er_f = orbit.ER(time[71])
    Ez_f = orbit.Ez(time[71])
    Lz_f = orbit.Lz(time[71])

    df['E_i'] = E_i
    df['ER_i'] = Er_i
    df['Ez_i'] = Ez_i
    df['Lz_i'] = Lz_i

    df['E_f'] = E_f
    df['ER_f'] = Er_f
    df['Ez_f'] = Ez_f
    df['Lz_f'] = Lz_f

    df['jp'] = orbit.jp(pot = potential)
    df['jr'] = orbit.jr(pot = potential)
    df['jz'] = orbit.jz(pot = potential)
    df['Op'] = orbit.Op(pot = potential)
    df['Or'] = orbit.Or(pot = potential)
    df['Oz'] = orbit.Oz(pot = potential)
    return df

In [121]:
df_heiles = compute_orbit(df_heiles, time)
df_split = compute_orbit(df_split, time)
df_young = compute_orbit(df_young, time)
df_older = compute_orbit(df_older, time)
df_rw = compute_orbit(df_rw, time)

df_heiles.to_csv('/Users/cam/Desktop/astro_research/radcliffe/clusters_integrated/df_heiles_orbit.csv', index = False)
df_split.to_csv('/Users/cam/Desktop/astro_research/radcliffe/clusters_integrated/df_split_orbit.csv', index = False)
df_young.to_csv('/Users/cam/Desktop/astro_research/radcliffe/clusters_integrated/df_maria_young_orbit.csv', index = False)
df_older.to_csv('/Users/cam/Desktop/astro_research/radcliffe/clusters_integrated/df_maria_older_orbit.csv', index = False)
df_rw.to_csv('/Users/cam/Desktop/astro_research/radcliffe/clusters_integrated/df_rw_orbit.csv', index = False)